In [ ]:
pip install openpyxl

# Imports

In [1]:
import spynnaker8 as p
import time
from matplotlib import pyplot as plt
import numpy as np
import spynnaker8.spynnaker_plotting as pl
import spynnaker8.utilities.neo_convertor as convert
from pyNN.utility.plotting import Figure, Panel
import os
import csv
import openpyxl
import random
import itertools

Detected PyNN version 0.9.4 and Neo version 0.6.1


# Loading Data

In [2]:
#Moving into the directory which contains the batch files
os.chdir("/home/jovyan/SpeechRecognition/Batch_files") #Create a bacth folder 
file_names = os.listdir()

#extracting the labels corresponding to each feature vector
list_of_numbers_in_batch_files = {}
for i in range(len(file_names)):
    list_to_append = []
    if(file_names[i][-11:] == "result.xlsx"):
        print(file_names[i])
        book = openpyxl.load_workbook(file_names[i])
        sheet = book.active
        row_count = sheet.max_row
        for j in range(1,row_count):
#             print(int(sheet.cell(row=j, column=2).value))
            list_to_append.append(int(sheet.cell(row=j, column=2).value))
        list_of_numbers_in_batch_files[file_names[i][:6]] = list_to_append
        
#extracting the data corresponding to each label per batch file
list_of_batch_files_data = {}
for i in range(len(file_names)):
    list_to_append = []
    if(file_names[i][-5:] == ".xlsx" and file_names[i][6] != "_"):
        print(file_names[i])
        book = openpyxl.load_workbook(file_names[i])
        sheet = book.active
        row_count = sheet.max_row
        col_count = sheet.max_column
        
        for j in range(1,row_count+1):
            list_of_data = []
            for k in range(2,col_count+1):
                list_of_data.append(sheet.cell(row=j, column=k).value)
            list_to_append.append(list_of_data)
        list_of_batch_files_data[file_names[i][:6]] = list_to_append

batch1_result.xlsx
batch8_result.xlsx
batch2_result.xlsx
batch6_result.xlsx
batch4_result.xlsx
batch3_result.xlsx
batch7_result.xlsx
batch5_result.xlsx
batch4.xlsx
batch6.xlsx
batch8.xlsx
batch2.xlsx
batch3.xlsx
batch5.xlsx
batch1.xlsx
batch7.xlsx


# Network

In [ ]:
#TEST ON SINGLE FILE

start_time = time.time()

TotalDuration = 100.0

current_pulse = list_of_batch_files_data['batch1'][0]
correct_digit = list_of_numbers_in_batch_files['batch1'][0]

# NumYCells = 1
NumYCells = 200
NumZCells = 1

#defining conductance based Izhikevich model
model_Izh = p.extra_models.Izhikevich_cond

snr_a=0.02 #0.005
snr_b=0.2 #0.32
snr_c=-65 #-65
snr_d=8 #2
snr_v_init = 30 #-70
snr_u_init = -13 #snr_b * snr_v_init

tau_ampa = 6###excitatory synapse time constant
tau_gabaa= 4### inhibitory synapse time constant
E_ampa = 0.0
E_gabaa = -80.0
current_bias = -2.
cell_params_input = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                   'v': snr_v_init, 'u': snr_u_init,
                   'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                   'i_offset': current_pulse,
                   'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                   }
cell_params_output = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                   'v': snr_v_init, 'u': snr_u_init,
                   'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                   'i_offset': 4.0,
                   'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                   }


p.setup(timestep=0.1, min_delay=1.0, max_delay=1.0)

z_population = []


#input population of 200
y_population = p.Population(NumYCells, model_Izh(**cell_params_input), label='RS_Izh_neuron_input')

#output population of 10
for j in range(10):
    z_population.append(p.Population(NumZCells, model_Izh(**cell_params_output), label='RS_Izh_neuron_output'))


ee_connector = p.AllToAllConnector()


stdp_projection = []

for j in range(10):

    if j==correct_digit: #if correct digit
        #defining Hebbian learning rule 
        timing_rule = p.SpikePairRule(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)
    else:
        #defining Anti-Hebbian learning rule
        timing_rule = p.SpikePairRuleAntiHebbian(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)

    #defining the STDP learning rule with the required parameters
    stdp_model = p.STDPMechanism(timing_dependence=timing_rule,weight_dependence=p.AdditiveWeightDependence(w_min=0.001,w_max=0.01),weight=random.uniform(0.001,0.01),delay=0.1)
    stdp_projection.append(p.Projection(y_population,z_population[j],p.OneToOneConnector(),synapse_type=stdp_model))



        
#simulation and results
y_population.record(['v', 'spikes'])
z_population[0].record(['v', 'spikes'])
z_population[1].record(['v', 'spikes'])
z_population[2].record(['v', 'spikes'])
z_population[3].record(['v', 'spikes'])
z_population[4].record(['v', 'spikes'])
z_population[5].record(['v', 'spikes'])
z_population[6].record(['v', 'spikes'])
z_population[7].record(['v', 'spikes'])
z_population[8].record(['v', 'spikes'])
z_population[9].record(['v', 'spikes'])

p.run(TotalDuration)


for i in range(len(stdp_projection)):
    print("Weights:{}".format(stdp_projection[i].get('weight', 'list')))


pre_spikes = y_population.get_data('spikes')
post_spikes = []
for i in range(10):
    post_spikes.append(z_population[i].get_data('spikes'))
# post_spikes = z_population[0].get_data('spikes')

Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[1].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[2].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[3].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[4].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[5].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[6].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[7].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[8].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[9].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), title="Speech Classifier", annotations="simulated with {}".format(p.name())).save('SpeechClassification.png')

# Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)))

p.end()        

In [ ]:
#NETWORK FOR TRAINING ONE BATCH FILE WITH WEIGHTS SAVING

start_time = time.time()

TotalDuration = 10000.0

features_list = list_of_batch_files_data['batch1']
digit_list = list_of_numbers_in_batch_files['batch1']

weights_list = []

#initialising random weights for the first iteration
for j in range(10):
    weights_list.append(random.uniform(0.001,0.01))

# yet to take weights and feed it to the network for the next feature vector
for (feature, correct_digit) in zip(features_list,digit_list):

    print("******************************************************")
    print("correct_digit", correct_digit)
    print("feature", feature)
    print("******************************************************")
    
    NumYCells = 200
    NumZCells = 1

    #defining conductance based Izhikevich model
    model_Izh = p.extra_models.Izhikevich_cond

    snr_a=0.02 #0.005
    snr_b=0.2 #0.32
    snr_c=-65 #-65
    snr_d=8 #2
    snr_v_init = 30 #-70
    snr_u_init = -13 #snr_b * snr_v_init

    tau_ampa = 6###excitatory synapse time constant
    tau_gabaa= 4### inhibitory synapse time constant
    E_ampa = 0.0
    E_gabaa = -80.0
    current_bias = -2.
    cell_params_input = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': feature,
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }
    cell_params_output = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': 4.0,
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }


    p.setup(timestep=0.1, min_delay=1.0, max_delay=1.0)
    
    z_population = []


    #input population of 200
    y_population = p.Population(NumYCells, model_Izh(**cell_params_input), label='RS_Izh_neuron_input')

    #output population of 10
    for j in range(10):
        z_population.append(p.Population(NumZCells, model_Izh(**cell_params_output), label='RS_Izh_neuron_output'))

    ee_connector = p.AllToAllConnector()


    stdp_projection = []

    for j in range(10):
        
        if j==correct_digit: #if correct digit
            #defining Hebbian learning rule 
            timing_rule = p.SpikePairRule(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)
        else:
            #defining Anti-Hebbian learning rule
            timing_rule = p.SpikePairRuleAntiHebbian(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)

        #defining the STDP learning rule with the required parameters
        stdp_model = p.STDPMechanism(timing_dependence=timing_rule,weight_dependence=p.AdditiveWeightDependence(w_min=0.001,w_max=0.01),weight=weights_list,delay=0.1)
        stdp_projection.append(p.Projection(y_population,z_population[j],p.OneToOneConnector(),synapse_type=stdp_model))

    #simulation and results
    y_population.record(['v', 'spikes'])
    z_population[0].record(['v', 'spikes'])
    z_population[1].record(['v', 'spikes'])
    z_population[2].record(['v', 'spikes'])
    z_population[3].record(['v', 'spikes'])
    z_population[4].record(['v', 'spikes'])
    z_population[5].record(['v', 'spikes'])
    z_population[6].record(['v', 'spikes'])
    z_population[7].record(['v', 'spikes'])
    z_population[8].record(['v', 'spikes'])
    z_population[9].record(['v', 'spikes'])

    p.run(TotalDuration)

    #redifine weights_list to get new values so that they can be used in the next iteration
    weights_list = []
    for i in range(len(stdp_projection)):
        print("Weights:{}".format(stdp_projection[i].get('weight', 'list')))
        weights_list.append(stdp_projection[i].get('weight', 'list')[0][2])

    pre_spikes = y_population.get_data('spikes')
    post_spikes = []
    for i in range(10):
        post_spikes.append(z_population[i].get_data('spikes'))
        
    # post_spikes = z_population[0].get_data('spikes')

#     Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[1].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[2].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[3].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[4].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[5].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[6].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[7].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[8].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[9].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), title="Speech Classifier", annotations="simulated with {}".format(p.name())).save('SpeechClassification.png')

    # Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)))

    p.end()  
    
    print("**************************************************")
    print(weights_list)
    print("**************************************************")


2020-04-17 10:30:06 INFO: Read cfg files: /home/jovyan/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/jovyan/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/jovyan/.spynnaker.cfg
2020-04-17 10:30:06 INFO: Will search these locations for binaries: /home/jovyan/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/common_model_binaries : /home/jovyan/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries
2020-04-17 10:30:06 WARNING: /home/jovyan/SpeechRecognition/Batch_files/reports has 7 old reports that have not been closed
2020-04-17 10:30:06 WARNING: /home/jovyan/SpeechRecognition/Batch_files/application_generated_data_files has 7 old reports that have not been closed
2020-04-17 10:30:06 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2020-04-17 10:30:06 

******************************************************
correct_digit 1
feature [8.101221634, 4.137519445, 4.000611559, 4.001506709, 4.000355698, 12.4460546, 4.137893178, 4.004002773, 4.000277956, 4.000399585, 12.12747706, 4.28776248, 4.001517548, 4.001083162, 4.001297226, 12.21164598, 4.479814506, 4.002333472, 4.001038458, 4.007318239, 12.11227067, 4.464589245, 4.007844483, 4.001694523, 4.007766104, 10.99426266, 4.647745991, 4.016264909, 4.001130293, 4.008557244, 12.37614753, 5.180199203, 4.026006687, 4.004185978, 4.00673913, 14.56310255, 5.980912912, 4.036846318, 4.001289231, 4.008189766, 20.95602039, 6.38863539, 4.043689934, 4.004242187, 4.014376272, 21.74626258, 6.297889587, 4.074861574, 4.013605049, 4.014053, 15.88188272, 6.412606911, 4.476410178, 4.019831373, 4.030281632, 14.92021285, 5.93133395, 5.41752714, 4.047578559, 4.077721122, 14.76182175, 5.936151966, 7.141159799, 4.039281891, 4.268887563, 15.817374, 6.341711525, 9.896180681, 4.068902598, 5.362363875, 14.5028866, 6.0992759

2020-04-17 10:30:06 INFO: Starting execution process
2020-04-17 10:30:06 INFO: Simulating for 100000 0.1ms timesteps using a hardware timestep of 10us
2020-04-17 10:30:17 INFO: Time 0:00:10.219479 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-04-17 10:30:31 INFO: Time 0:00:14.669997 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-04-17 10:30:58 INFO: Time 0:00:26.598943 taken by PartitionAndPlacePartitioner
Created spalloc job 5337141
2020-04-17 10:30:58 INFO: Created spalloc job 5337141
Waiting for board power commands to complete.
2020-04-17 10:30:58 INFO: Waiting for board power commands to complete.
2020-04-17 10:31:07 INFO: Time 0:00:08.959975 taken by SpallocAllocator
2020-04-17 

Weights:[(0, 0, 0.00296116)]
Weights:[(0, 0, 0.00607586)]


Getting ['source', 'target', 'weight']s for projection between RS_Izh_neuron_input and RS_Izh_neuron_output2
|0%                          50%                         100%|
Getting ['source', 'target', 'weight']s for projection between RS_Izh_neuron_input and RS_Izh_neuron_output2
|0%                          50%                         100%|
Getting ['source', 'target', 'weight']s for projection between RS_Izh_neuron_input and RS_Izh_neuron_output3
|0%                          50%                         100%|

...

Getting ['source', 'target', 'weight']s for projection between RS_Izh_neuron_input and RS_Izh_neuron_output9
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_input
|0%                          50%                         100%|


Weights:[(0, 0, 0.00099993)]


Getting spikes for RS_Izh_neuron_output
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output1
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output2
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output3
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output4
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output5
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output6
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output7
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output8
|0%                          50%                         100%|
Getting spikes for RS_Izh_neuron_output9
|0%                     

**************************************************
[0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531, 0.0009999275207519531]
**************************************************
******************************************************
correct_digit 3
feature [4.001591834, 4.005146197, 4.00040493, 4.008143039, 4.011339005, 4.000851455, 4.000407998, 4.0000279, 4.000684523, 4.002528843, 4.000140133, 4.000039117, 4.000013636, 4.000582124, 4.000556139, 4.000381985, 4.000021962, 4.000004434, 4.000707262, 4.000265008, 4.000059808, 4.000073498, 4.000041675, 4.000205904, 4.000208627, 4.000153435, 4.000034062, 4.000018756, 4.00037268, 4.000099548, 4.000055423, 4.000036795, 4.000022418, 4.000739329, 4.000046107, 4.00408777, 4.000283631, 4.000043044, 4.000197569, 4.000050976, 4.072311224, 4.005723005, 4.000289447, 4.000607946, 4.000020234, 4.21506

2020-04-17 13:30:19 WARNING: Formal PyNN specifies that u should be set using initial_values not cell_params
2020-04-17 13:30:19 INFO: Starting execution process
2020-04-17 13:30:19 INFO: Simulating for 100000 0.1ms timesteps using a hardware timestep of 10us
2020-04-17 13:30:25 INFO: Time 0:00:05.606487 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-04-17 13:32:14 INFO: Time 0:01:48.906806 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-04-17 13:32:25 INFO: Time 0:00:11.633844 taken by PartitionAndPlacePartitioner
Created spalloc job 5337724
2020-04-17 13:32:26 INFO: Created spalloc job 5337724
Waiting for board power commands to complete.
2020-04-17 13:32:26 INFO: Waiting for board po

In [ ]:
#NETWORK FOR TRAINING ALL BATCH FILES WITH WEIGHTS SAVING

batch_names = ['batch1','batch2','batch3','batch4','batch5','batch6','batch7','batch8','batch9']

weights_list = []

#initialising random weights for the first iteration
for j in range(10):
    weights_list.append(random.uniform(0.01,0.1))

for batch in batch_names:
    
    start_time = time.time()

    TotalDuration = 100.0

    features_list = list_of_batch_files_data[batch]
    digit_list = list_of_numbers_in_batch_files[batch]

    for (feature, correct_digit) in zip(features_list,digit_list):
        
        print("**************************************************************************")
        print(batch)
        print("**************************************************************************")

        NumYCells = 200
        NumZCells = 1

        #defining conductance based Izhikevich model
        model_Izh = p.extra_models.Izhikevich_cond

        snr_a=0.02 #0.005
        snr_b=0.2 #0.32
        snr_c=-65 #-65
        snr_d=8 #2
        snr_v_init = 30 #-70
        snr_u_init = -13 #snr_b * snr_v_init

        tau_ampa = 6###excitatory synapse time constant
        tau_gabaa= 4### inhibitory synapse time constant
        E_ampa = 0.0
        E_gabaa = -80.0
        current_bias = 0.
        cell_params_input = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                           'v': snr_v_init, 'u': snr_u_init,
                           'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                           'i_offset': feature,
                           'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                           }
        cell_params_output = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                           'v': snr_v_init, 'u': snr_u_init,
                           'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                           'i_offset': 4.0,
                           'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                           }


        p.setup(timestep=0.1, min_delay=1.0, max_delay=1.0)

        z_population = []


        #input population of 200
        y_population = p.Population(NumYCells, model_Izh(**cell_params_input), label='RS_Izh_neuron_input')

        #output population of 10
        for j in range(10):
            z_population.append(p.Population(NumZCells, model_Izh(**cell_params_output), label='RS_Izh_neuron_output'))

        ee_connector = p.AllToAllConnector()


        stdp_projection = []

        for j in range(10):

            if j==correct_digit: #if correct digit
                #defining Hebbian learning rule 
                timing_rule = p.SpikePairRule(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)
            else:
                #defining Anti-Hebbian learning rule
                timing_rule = p.SpikePairRuleAntiHebbian(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)

            #defining the STDP learning rule with the required parameters
            stdp_model = p.STDPMechanism(timing_dependence=timing_rule,weight_dependence=p.AdditiveWeightDependence(w_min=0.01,w_max=0.1),weight=weights_list,delay=0.1)
            stdp_projection.append(p.Projection(y_population,z_population[j],p.OneToOneConnector(),synapse_type=stdp_model))

        #simulation and results
        y_population.record(['v', 'spikes'])
        z_population[0].record(['v', 'spikes'])
        z_population[1].record(['v', 'spikes'])
        z_population[2].record(['v', 'spikes'])
        z_population[3].record(['v', 'spikes'])
        z_population[4].record(['v', 'spikes'])
        z_population[5].record(['v', 'spikes'])
        z_population[6].record(['v', 'spikes'])
        z_population[7].record(['v', 'spikes'])
        z_population[8].record(['v', 'spikes'])
        z_population[9].record(['v', 'spikes'])

        p.run(TotalDuration)

        #redifine weights_list to get new values so that they can be used in the next iteration
        weights_list = []
        for i in range(len(stdp_projection)):
            print("Weights:{}".format(stdp_projection[i].get('weight', 'list')))
            weights_list.append(stdp_projection[i].get('weight', 'list')[0][2])
            
        print("**************************************************************************")
        print(weights_list)
        print("**************************************************************************")


        pre_spikes = y_population.get_data('spikes')
        post_spikes = []
        for i in range(10):
            post_spikes.append(z_population[i].get_data('spikes'))

        # post_spikes = z_population[0].get_data('spikes')

    #     Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[1].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[2].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[3].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[4].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[5].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[6].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[7].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[8].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[9].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), title="Speech Classifier", annotations="simulated with {}".format(p.name())).save('SpeechClassification.png')

        # Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)))

        p.end()     

In [ ]:
1:[0.050235748291015625, 0.049617767333984375, 0.050235748291015625, 0.050235748291015625, 0.050235748291015625, 0.050235748291015625, 0.050235748291015625, 0.050235748291015625, 0.050235748291015625, 0.050235748291015625]
5:[0.050510406494140625, 0.050510406494140625, 0.050510406494140625, 0.050510406494140625, 0.050510406494140625, 0.049961090087890625, 0.050510406494140625, 0.050510406494140625, 0.050510406494140625, 0.050510406494140625]
1:[0.050716400146484375, 0.050304412841796875, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375]
2:[0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375]
[0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375, 0.050716400146484375]
2:[0.05074310302734375, 0.05074310302734375, 0.050689697265625, 0.05074310302734375, 0.05074310302734375, 0.05074310302734375, 0.05074310302734375, 0.05074310302734375, 0.05074310302734375, 0.05074310302734375]

 
    
    
    
    
    
    
    